In [1]:
import pandas as pd
import numpy as np
from glob import glob
import numbers

In [2]:
filenames = glob('Combined_*[a,b,c,d,e,f,g,h,i,j,k]*160c.dta')
list_dfs  = [pd.read_stata(f) for f in filenames]

In [4]:
list_dfs = [list_dfs[i].set_index('xwaveid') for i in range(11)]

In [7]:
male, female = '[1] Male', '[2] Female'
FT, PT = '[1] Employed FT', '[2] Employed PT'

In [16]:
def table(sex, employ):
    list_males = [list_dfs[i][list_dfs[i].filter(regex='hgsex$').columns]for i in range(11)]
    list_males = [list_dfs[i][list_males[i].iloc[:,0] == sex] for i in range(11)]

    waves_employed_index = [list_dfs[i].filter(regex='esdtl$').columns for i in range(11)] #
    waves_employed = [list_dfs[i][waves_employed_index[i]] for i in range(11)] #waves_employed[0-10] are now created (waves1-11)
    
    if (employ != None):
        males_employed = [list_males[i][waves_employed[i].iloc[:,0] == employ] for i in range(11)]
    else:
        males_employed = list_males

    pref1 = [males_employed[i].filter(regex='jbhrcpr$') for i in range(11)]
    pref2 = [males_employed[i].filter(regex='jbhrcpr$') for i in range(11)]
    pref2 = pref2[1:]

    diff = [pref1[i].join(pref2[i]) for i in range(10)]
    diff1 = [diff[i].iloc[:,1].cat.add_categories('Missing') for i in range(10)]
    diff1 = [diff1[i].fillna('Missing') for i in range(10)]

    diff = [pd.DataFrame(diff[i].iloc[:,0]).join(pd.DataFrame(diff1[i])) for i in range(10)]

    clean_fewer = [diff[i][diff[i].iloc[:,0] == '[1] Fewer hours'] for i in range(10)]
    clean_fewer = [clean_fewer[i].iloc[:,1].value_counts(normalize=True) for i in range(10)]

    clean_current = [diff[i][diff[i].iloc[:,0] == '[2] About the same']for i in range(10)]
    clean_current = [clean_current[i].iloc[:,1].value_counts(normalize=True) for i in range(10)]

    clean_more = [diff[i][diff[i].iloc[:,0] == '[3] More hours'] for i in range(10)]
    clean_more = [clean_more[i].iloc[:,1].value_counts(normalize=True) for i in range(10)]

    length = []
    for i in range(11):
        pref = (pref1[i].iloc[:,0] == '[1] Fewer hours').value_counts()
        pref = pref[1]
        length.append(pref)

    len1=np.array(length).sum()

    length = []
    for i in range(11):
        pref = (pref1[i].iloc[:,0] == '[2] About the same').value_counts()
        pref = pref[1]
        length.append(pref)

    len2=np.array(length).sum()

    length = []
    for i in range(11):
        pref = (pref1[i].iloc[:,0] == '[3] More hours').value_counts()
        pref = pref[1]
        length.append(pref)

    len3=np.array(length).sum()

    diff = [pref1[i].join(pref2[i]) for i in range(10)]
    diff1 = [diff[i].iloc[:,1].cat.add_categories('Missing') for i in range(10)]
    diff1 = [diff1[i].fillna('Missing') for i in range(10)]
    diff = [pd.DataFrame(diff[i].iloc[:,0]).join(pd.DataFrame(diff1[i])) for i in range(10)]

    cleaned = diff
    clean_fewer = [cleaned[i][cleaned[i].iloc[:,0] == '[1] Fewer hours'] for i in range(10)]
    clean_fewer = [clean_fewer[i].iloc[:,1].value_counts(normalize=True) for i in range(10)]

    clean_current = [cleaned[i][cleaned[i].iloc[:,0] == '[2] About the same']for i in range(10)]
    clean_current = [clean_current[i].iloc[:,1].value_counts(normalize=True) for i in range(10)]

    clean_more = [cleaned[i][cleaned[i].iloc[:,0] == '[3] More hours'] for i in range(10)]
    clean_more = [clean_more[i].iloc[:,1].value_counts(normalize=True) for i in range(10)]

    clean_fewer = pd.DataFrame(clean_fewer)
    clean_current = pd.DataFrame(clean_current)
    clean_more  = pd.DataFrame(clean_more)

    clean_unfin = clean_fewer.merge(clean_current, left_index=True, right_index=True)
    clean_merged = clean_unfin.merge(clean_more, left_index=True, right_index=True)
    unclean = clean_merged.mean(axis=0)
    unclean = unclean*100

    unclean = pd.DataFrame(unclean)

    fewer_hrs = unclean[:8]
    curr_hrs = unclean[8:16]
    more_hrs = unclean[16:]

    fewer_hrs.columns = ['Prefer to work fewer hours (t-1)']
    curr_hrs.columns = ['Current hours are good (t-1)']
    more_hrs.columns = ['Prefer to work more hours (t-1)']

    fewer_hrs.index = more_hrs.index
    curr_hrs.index = more_hrs.index

    final_table = fewer_hrs.join(curr_hrs)
    final_table = final_table.join(more_hrs)

    final_table.sort_values('Prefer to work fewer hours (t-1)', ascending=False, inplace=True)

    final_table.rename(index={'[1] Fewer hours':'Prefer to work less',
                          '[2] About the same':'Current hours okay',
                          '[3] More hours':'Prefer to work more',
                          'Missing':'Other',
                          '[-1] Not asked':'Other',
                         '[-10] Non-responding person':'Other',
                         '[-3] Dont know':'Other',
                         '[-4] Refused/Not stated':'Other'}, inplace=True)
    final_table.rename(columns = {'Prefer to work fewer hours (t-1)':'Prefer to work less (%)',
                              'Current hours are good (t-1)':'Current hours okay (%)',
                              'Prefer to work more hours (t-1)':'Prefer to work more (%)'}, inplace=True)
    final_table.round(2)
    final_table = final_table.groupby(level=0).sum()
    final_table.sort_values(by='Prefer to work less (%)',ascending=False)

    final_table.index.name = ''

    final_table.rename(index={'Other':'Missing from t+1'})

    Number_of_Observations = {'Prefer to work less (%)': len1, 'Current hours okay (%)': len2, 'Prefer to work more (%)': len3}
    final_table = final_table.append(Number_of_Observations, ignore_index=True)
    final_table.index = ['Current hours okay', 'Missing from t+1', 'Prefer to work less', 'Prefer to work more', 'Number of observations']

    final_table=final_table.round(2)

    return final_table

In [17]:
table(male, None) #Table 6.1

,Prefer to work less (%),Current hours okay (%),Prefer to work more (%)
Current hours okay,30.51,62.32,36.95
Missing from t+1,12.12,14.38,21.43
Prefer to work less,54.21,16.07,7.27
Prefer to work more,3.17,7.24,34.36
Number of observations,14241.00,28341.00,6791.00


In [18]:
table(male, FT) #Table 6.2

C:\Users\idias\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


,Prefer to work less (%),Current hours okay (%),Prefer to work more (%)
Current hours okay,28.53,61.72,38.55
Missing from t+1,15.02,16.07,22.42
Prefer to work less,54.79,17.47,8.39
Prefer to work more,1.66,4.75,30.64
Number of observations,13770.00,23711.00,3387.00


In [19]:
table(male, PT) #Table 6.3

C:\Users\idias\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


,Prefer to work less (%),Current hours okay (%),Prefer to work more (%)
Current hours okay,30.01,43.14,16.55
Missing from t+1,46.57,42.18,53.69
Prefer to work less,14.92,3.46,0.90
Prefer to work more,8.51,11.22,28.87
Number of observations,469.00,4616.00,3397.00


In [20]:
table(female,None) #Table 7.1

,Prefer to work less (%),Current hours okay (%),Prefer to work more (%)
Current hours okay,31.53,59.80,36.16
Missing from t+1,15.29,16.54,22.45
Prefer to work less,49.22,14.85,7.20
Prefer to work more,3.95,8.82,34.18
Number of observations,11338.00,26020.00,7351.00


In [21]:
table(female,FT) #Table 7.2

C:\Users\idias\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


,Prefer to work less (%),Current hours okay (%),Prefer to work more (%)
Current hours okay,22.09,54.13,39.37
Missing from t+1,26.13,23.92,36.86
Prefer to work less,51.05,19.74,6.34
Prefer to work more,0.73,2.21,17.42
Number of observations,9403.00,12408.00,923.00


In [22]:
table(female,PT) #Table 7.3

C:\Users\idias\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


,Prefer to work less (%),Current hours okay (%),Prefer to work more (%)
Current hours okay,37.85,52.44,24.11
Missing from t+1,33.76,30.52,40.44
Prefer to work less,21.27,5.32,2.43
Prefer to work more,7.13,11.74,33.02
Number of observations,1933.00,13593.00,6410.00


In [77]:
final_table.round(2)

,Prefer to work less (%),Current hours okay (%),Prefer to work more (%)
Other,46.35,42.06,53.66
Current hours okay,30.01,43.14,16.55
Prefer to work less,14.92,3.46,0.90
Prefer to work more,8.51,11.22,28.87
Other,0.22,0.12,0.03
Other,0.00,0.00,0.00
Other,0.00,0.00,0.00
Other,0.00,0.00,0.00


,Prefer to work less (%),Current hours okay (%),Prefer to work more (%)
Other,46.565525,42.184461,53.689278
Current hours okay,30.010749,43.136277,16.545957
Prefer to work less,14.915645,3.459351,0.898678
Prefer to work more,8.508080,11.219911,28.866087


Small extension: The above table is an average of ALL waves (t-1). 

,Prefer to work less (%),Current hours okay (%),Prefer to work more (%)
,,,
Current hours okay,30.010749,43.136277,16.545957
Missing from t+1,46.565525,42.184461,53.689278
Prefer to work less,14.915645,3.459351,0.898678
Prefer to work more,8.508080,11.219911,28.866087


In [85]:
final_table

,Prefer to work less (%),Current hours okay (%),Prefer to work more (%)
0,30.010749,43.136277,16.545957
1,46.565525,42.184461,53.689278
2,14.915645,3.459351,0.898678
3,8.508080,11.219911,28.866087
4,469.000000,4616.000000,3397.000000


,Prefer to work less (%),Current hours okay (%),Prefer to work more (%)
Current hours okay,30.01,43.14,16.55
Missing from t+1,46.57,42.18,53.69
Prefer to work less,14.92,3.46,0.90
Prefer to work more,8.51,11.22,28.87
Number of observations,469.00,4616.00,3397.00
